In [1]:
import numpy as np


# Generating the A matrix (system interaction matrix)

In [2]:
A=np.array([
    [1,1,0],
    [0,1,1],
    [1,1,1],
    [2,2,0]
], dtype=float)

In [3]:
rank_A=np.linalg.matrix_rank(A)
print("Rank of A:", rank_A)

Rank of A: 3


Hence A is a rank-deficient matrix.

# Three physical quantities observed by four sensors

In [4]:
#Ground truth
x_true=np.array([3,4,3], dtype=float)

In [5]:
#Clean measurements
b_clean=A @ x_true

In [6]:
#Correlated noise covariance
Sigma=np.array([
    [0.2, 0.1, 0.0, 0.05],
    [0.1, 0.2, 0.1, 0.0 ],
    [0.0, 0.1, 0.2, 0.1 ],
    [0.05, 0.0, 0.1, 0.2 ]
])

In [7]:
# Generate correlated noise
noise = np.random.multivariate_normal(
    mean=np.zeros(4),
    cov=Sigma
)


In [8]:
# Noisy measurements
b = b_clean + noise

Thus b is never equal to A @ x_true.

In [9]:
U, S, Vt = np.linalg.svd(A)

tol = 1e-10 #tolerance for singular values
null_space = Vt.T[:, S < tol]
print("Null space of A:", null_space)

Null space of A: []


In [10]:
cond_A = np.linalg.cond(A)
print("Condition number of A:", cond_A)
print("Condition number from singular values:", S[0] / S[-1])


Condition number of A: 7.866945653318726
Condition number from singular values: 7.866945653318725


In [11]:
AtA = A.T @ A

eigvals_AtA, eigvecs_AtA = np.linalg.eig(AtA)

print("Eigenvalues of A^T A:")
print(eigvals_AtA)

print("\nEigenvectors of A^T A:")
print(eigvecs_AtA)


Eigenvalues of A^T A:
[12.94392154  0.20914793  1.84693053]

Eigenvectors of A^T A:
[[-0.65727018 -0.62554549 -0.42034361]
 [-0.72847433  0.67028426  0.14157742]
 [-0.19318659 -0.39926415  0.89625169]]


In [12]:
AAt = A @ A.T

eigvals_AAt, eigvecs_AAt = np.linalg.eig(AAt)

print("Eigenvalues of A A^T:")
print(eigvals_AAt)

print("\nEigenvectors of A A^T:")
print(eigvecs_AAt)


Eigenvalues of A A^T:
[ 1.29439215e+01  1.84693053e+00 -4.57678508e-16  2.09147931e-01]

Eigenvectors of A A^T:
[[-3.85168029e-01 -2.05123216e-01 -8.94427191e-01 -9.78266597e-02]
 [-2.56175880e-01  7.63660919e-01  7.36049830e-16 -5.92617852e-01]
 [-4.38864289e-01  4.54361455e-01 -3.70689790e-16  7.75212103e-01]
 [-7.70336059e-01 -4.10246432e-01  4.47213595e-01 -1.95653319e-01]]


In [13]:
#SVD
AtA = A.T @ A
print(AtA)
eigvals, eigvecs = np.linalg.eig(AtA)

# sort in descending order
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

print("Eigenvalues:", eigvals)
print("Eigenvectors:\n", eigvecs)


[[6. 6. 1.]
 [6. 7. 2.]
 [1. 2. 2.]]
Eigenvalues: [12.94392154  1.84693053  0.20914793]
Eigenvectors:
 [[-0.65727018 -0.42034361 -0.62554549]
 [-0.72847433  0.14157742  0.67028426]
 [-0.19318659  0.89625169 -0.39926415]]


In [15]:
singular_values = np.sqrt(eigvals)
print("Singular values:", singular_values)
V = eigvecs
U = np.zeros((A.shape[0], len(singular_values)))

for i in range(len(singular_values)):
    U[:, i] = (A @ V[:, i]) / singular_values[i]
U = U / np.linalg.norm(U, axis=0)
Sigma = np.diag(singular_values)

for i in range(len(singular_values)):
    Sigma[i, i] = singular_values[i]

print("Sigma:\n", Sigma)
A_reconstructed = U @ Sigma @ V.T
print("Reconstruction error:", np.linalg.norm(A - A_reconstructed))


Singular values: [3.59776619 1.35901822 0.45732694]
Sigma:
 [[3.59776619 0.         0.        ]
 [0.         1.35901822 0.        ]
 [0.         0.         0.45732694]]
Reconstruction error: 2.0539840288898652e-15


In [16]:
b_proj = U @ U.T @ b
residual = b - b_proj
print("A^T residual:", A.T @ residual)


A^T residual: [-2.04281037e-14  1.50990331e-14 -5.32907052e-15]


In [17]:
AtA = A.T @ A
Atb = A.T @ b

x_gauss = np.linalg.solve(AtA, Atb)
print("Gauss solution:", x_gauss)

Gauss solution: [3.06560836 3.91115056 3.02151929]


In [18]:
from scipy.linalg import lu_factor, lu_solve

lu, piv = lu_factor(AtA)
x_lu = lu_solve((lu, piv), Atb)
print("LU solution:", x_lu)

LU solution: [3.06560836 3.91115056 3.02151929]


In [19]:
Q, R = np.linalg.qr(A, mode='reduced')
x_qr = np.linalg.solve(R, Q.T @ b)
print("QR solution:", x_qr)

QR solution: [3.06560836 3.91115056 3.02151929]


In [20]:
x_pinv = np.linalg.pinv(A) @ b
print("Pseudoinverse solution:", x_pinv)

Pseudoinverse solution: [3.06560836 3.91115056 3.02151929]


In [21]:
def jacobi(A, b, x0, max_iter=1000, tol=1e-6):
    D = np.diag(np.diag(A))
    R = A - D
    x = x0.copy()
    for _ in range(max_iter):
        x_new = np.linalg.inv(D) @ (b - R @ x)
        if np.linalg.norm(x_new - x) < tol:
            break
        x = x_new
    return x

x_jacobi = jacobi(AtA, Atb, np.zeros(3))
print("Jacobi solution:", x_jacobi)

Jacobi solution: [-1.14924893e+83 -1.15186537e+83 -1.42765528e+83]


In [22]:
def gauss_seidel(A, b, x0, max_iter=1000, tol=1e-6):
    x = x0.copy()
    for _ in range(max_iter):
        x_old = x.copy()
        for i in range(len(b)):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i+1:], x_old[i+1:])
            x[i] = (b[i] - s1 - s2) / A[i, i]
        if np.linalg.norm(x - x_old) < tol:
            break
    return x

x_gs = gauss_seidel(AtA, Atb, np.zeros(3))
print("Gauss-Seidel solution:", x_gs)

Gauss-Seidel solution: [3.0656168  3.91114199 3.02152365]


In [23]:
noise = 0.01 * np.random.randn(4)
b_noisy = b + noise
x_qr_noisy   = np.linalg.solve(R, Q.T @ b_noisy)
x_pinv_noisy = np.linalg.pinv(A) @ b_noisy
print("QR solution with noise:", x_qr_noisy)
print("Pseudoinverse solution with noise:", x_pinv_noisy)

QR solution with noise: [3.0504962  3.92460427 3.01453278]
Pseudoinverse solution with noise: [3.0504962  3.92460427 3.01453278]
